### Открываю tmx-файл в Heartsome TMX Editor. Сначала просто ознакамливаюсь с данными. 

#### Устанавливаю модуль Python для работы с tmx-файлами

In [ ]:
!pip install translate-toolkit

In [1]:
from translate.storage.tmx import tmxfile

In [2]:
#Импортирую tmx файл

with open("testTMX.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'ru', 'eng')

In [4]:
# Функция для проверки юнитов только с цифрами и точками, нужна будет дальше

digitsdot = "1234567890."

def isdotdigit(string):
    for k in string:
        if k in digitsdot:
            for k in digitsdot:
                ;
        else:
            return False
    return True    

char = "1.3.2.3"

print(isdotdigit(char))

True


### Начинаю проверку датасета на пустоты,  на юниты, не имеющие ценности и т.д.

In [10]:
s = 0
t = 0
b = 0
count = 0
dd = 0
trash = 0

for node in tmx_file.unit_iter():
    count += 1
    source = node.getid()
    target = node.gettarget()
    
# Проверяем, есть ли в юните пустой source, либо пустой target

    if node.isblank():
        b += 1
        
# Проверяем, является ли source, либо target, исключительно числом    

    if source.isnumeric():
        s += 1
    if target.isnumeric():
        t += 1
        
# Предположим, что все source, в которых меньше, чем 6 символов, не имеют никакой ценности. 
# Посмотрим на такие юниты и посчитаем количество
# Из этих юнитов даже беглым взглядом можно вычленить много мусора: юниты-числа с точками, буквы(не слова) с точками и т.д.
# Их всего 72(1,8% от датасета)

    if len(source) < 6:
        print(source)
        print(target)
        print("________")
        trash += 1

# Проверим, возможно, есть множество юнитов больше 6 символов, где только цифры и точки, т.е. также не имеющие ценность

    if (isdotdigit(source)):
        dd += 1
                    
print("Пустых:", b)
print("Source-число:", s)
print("Target-число:", t)
print("Source меньше 6 символов:", trash)
print("В source только цифры и точки:", dd)
print("Количество юнитов:", count)

Пустых: 0
Source-число: 0
Target-число: 0
Source меньше 6 символов: 0
В source только цифры и точки: 0
Количество юнитов: 3968


#### Удаляем вычисленные невалидные юниты 

In [9]:
for node in tmx_file.getunits():
    source = node.getid()
    target = node.gettarget()
    
# Удаляем юниты с target-число   

    if target.isnumeric():
        tmx_file.removeunit(node)    

# Удаляем юниты с source меньше 6 символов

    elif (len(source) < 6):
        tmx_file.removeunit(node)
        
# Удаляем юниты только c цифрами и точками в source

    elif (isdotdigit(source)):
        tmx_file.removeunit(node)

#### Взяв список символов, валидных для русской и английской раскладки (включая space), я могу вычислить юниты, которые содержат порченые символы

In [15]:
count = 0
unknown = 0

_chars = u"~!@#$%^&qwertyuiop[]asdfghjkl;'zxcvbnm,./QWERTYUIOP{}ASDFGHJKL:\"|ZXCVBNM<>?ё!\"№;%:?йцукенгшщзхъфывапролджэячсмитьбю.ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭ/ЯЧСМИТЬБЮ, -1234567890()«—»–…+“”€*©"

for node in tmx_file.unit_iter():
    count += 1
    source = node.getid()
    target = node.gettarget()

    for s in source:
        if s not in _chars:
            print(s)
            print(source)
            print(target)
            print("-------------------------------------------------------------------")
            unknown += 1
            break;

print("Unknown:", unknown)
print("Количество юнитов:", count)            

Unknown: 0
Количество юнитов: 3861


In [14]:
#Их оказалось всего 107(2,7% от датасета), также удаляем их

for node in tmx_file.getunits():
    source = node.getid()
    
    for s in source:
        if s not in _chars:
            tmx_file.removeunit(node)
            break;

In [ ]:
#Увеличиваем видимость выводимых строк

pd.set_option('max_colwidth', 8000)